In [1]:
import tensorflow as tf
from object_detection.utils import dataset_util

import numpy as np
import cv2
from convenience_funtions import *
from scipy.spatial import distance as dist

flags = tf.app.flags
flags.DEFINE_string('output_path', '/home/siddiki/video2tfrecord/truck.tfrecords', 'Tf store directory')
FLAGS = flags.FLAGS

/home/siddiki/miniconda3/envs/TF/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
filename = 'Truck_Single_1.mp4'
cap = cv2.VideoCapture(filename)
# Parameters to write to video
out_filename = 'Truck_Single_1_Out' + ".avi"
out = cv2.VideoWriter(out_filename,cv2.VideoWriter_fourcc('M','J','P','G'), 5, (800,450))

ret, old_frame = cap.read()
old_frame = resize(old_frame, width=800)
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Parameters for Flow Calculation
move_step = 16
move_pix = 1.2 # How many pixels motion should be captured 
x_fac = 45 # x size of bounding box from center of object
y_fac = 30   # y size of bounding box from center of object
flow = None

In [3]:
def compute_bbox(current_frame, old_frame, flow):
    
    ## Parameters
    dict1 = {}
    numbers = []  
    final_bbox = []
    central_coord = []
    dist_px = []
    flow_val = 0
    central_x = 0
    central_y = 0
    
    
    flow = cv2.calcOpticalFlowFarneback(old_frame, current_frame, flow, pyr_scale=0.5, levels=3, winsize=15, iterations=3,
                                        poly_n=5, poly_sigma=1.5, flags=0)

    for y in range(0, flow.shape[0], move_step):

        for x in range(0, flow.shape[1],move_step):
            fx, fy = flow[y, x]

            if abs(fx) >=move_pix or abs(fy) >=move_pix:
                ##Store each point and its movement
                dict1[(x,y)] = fx+fy
#                 cv2.line(frame, (x, y), (int(x + fx), int(y + fy)), (255,0,0))

    for key in dict1.keys():
        numbers.append(key)

    ## Find clusters that are close together
    clusters = {}
    dIndex = 0
    for i in range(len(numbers) - 1):
        distance = dist.euclidean(numbers[i], numbers[i+1])
        if distance <= 120:
            if not dIndex in clusters.keys(): clusters[dIndex] = []
            clusters[dIndex].append(numbers[i])
            clusters[dIndex].append(numbers[i + 1])
            
        else:
            dIndex += 1
            
    ## Extract 1 bbox from each cluster

    def getkey(item):
        return item[0]

    #print("Num clusters: ", len(clusters))
    

    if len(clusters) <=5:
        for key in clusters:
            val = clusters[key]
            if len(val) >= 1:  # Multiple bbox
                # Sort the val list
                val = sorted(val, key=lambda element: (element[0], element[1]))
                #Find average values in the cluster
                for value in val:
                    flow_val += abs(dict1[value])
                    central_x += value[0]
                    central_y += value[1]
                flow_val =flow_val/len(val)
                central_x = int(central_x/len(val))
                central_y = int(central_y/len(val))

                # Store values in a dictionary
                dist_px.append(flow_val)
                central_coord.append((central_x, central_y))

                final_bbox.append((central_x-x_fac, central_x+x_fac, central_y-y_fac, central_y+y_fac))

    return final_bbox, flow
    

In [4]:
def writeVideo(frame, final_bbox):
    for i in range(len(final_bbox)):
        cv2.rectangle(frame, (final_bbox[i][0], final_bbox[i][2]), (final_bbox[i][1], final_bbox[i][3]), (0, 255, 0), 2)
    out.write(frame)

## TF Records Part

In [5]:
#float_list=tf.train.FloatList(value=value)
#float_list=tf.train.FloatList(value=[value])
'''
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
'''
def create_tf_example(image, Bbox):
  
  height = frame.shape[0] # Image height
  width = frame.shape[1] # Image width
  depth = frame.shape[2] # Image depth
    
  image = np.asarray(image)  
  encoded_image_data = image.tostring() # Encoded image bytes  

  xmins = [Bbox[0][0]/800] # List of normalized left x coordinates in bounding box (1 per box)
  xmaxs = [Bbox[0][1]/800] # List of normalized right x coordinates in bounding box
             # (1 per box)
  ymins = [Bbox[0][2]/450] # List of normalized top y coordinates in bounding box (1 per box)
  ymaxs = [Bbox[0][3]/450] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
  classes_text = [b'Truck'] # List of string class name of bounding box (1 per box)
  classes = [1] # List of integer class id of bounding box (1 per box)
  # TODO END
  tf_label_and_data = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/depth': dataset_util.int64_feature(depth),      
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),      
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
  }))
  return tf_label_and_data

In [6]:
writer = tf.python_io.TFRecordWriter(FLAGS.output_path)

while(1):
    ret,frame = cap.read()

    if not ret:
        break

    frame = resize(frame, width=800)
    current_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    bbox, flow = compute_bbox(old_gray, current_gray, flow)
    
    if not bbox:
        break
    
    writeVideo(frame, bbox)
    tf_example = create_tf_example(frame, bbox)
    writer.write(tf_example.SerializeToString())
    
    old_gray = current_gray.copy()
    
    

writer.close()    
cv2.destroyAllWindows()
cap.release()    